In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 1s 1us/step


In [3]:
# 读取并为 py2 compat 解码
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# 文本中的非重复字符
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [12]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [16]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024

In [17]:
# 由于后面的GRU层设定了stateful=True, 所以必须事先指定batch_size，
# 对于Sequential产生的模型，使用参数batch_input_shape=[batch_size, None]来指定
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]
                             ),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
model.trainable_weights[-2]

<tf.Variable 'dense/kernel:0' shape=(1024, 65) dtype=float32, numpy=
array([[ 3.7833877e-02,  2.7760267e-02, -2.2964410e-02, ...,
         2.1318175e-02,  3.1876273e-02, -5.4338869e-02],
       [-1.0660991e-02, -3.0338611e-02, -3.7181508e-02, ...,
         2.5233179e-03,  6.1126813e-02, -1.7367035e-02],
       [-2.0814110e-02,  4.0146604e-02, -1.9724749e-02, ...,
         6.3543215e-02,  4.5554072e-02,  2.7987942e-02],
       ...,
       [ 5.0770514e-02,  6.8314373e-05,  4.6781272e-02, ...,
        -2.6799582e-02, -6.6148557e-02, -5.5405870e-03],
       [ 4.9420126e-02,  1.6058870e-02,  9.0734586e-03, ...,
         1.2849569e-02,  7.3206410e-02, -4.5124173e-02],
       [ 1.9506685e-02,  5.8336303e-02,  6.9760680e-02, ...,
        -6.1879314e-02, -1.2855995e-02, -4.8698984e-02]], dtype=float32)>

In [21]:
a = next(iter(dataset.take(1)))

In [23]:
a

(<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
 array([[43,  1, 25, ..., 41, 56, 59],
        [13, 51, 43, ..., 63,  1, 21],
        [13, 52, 42, ..., 56, 52,  1],
        ...,
        [50,  1, 45, ..., 15, 53, 51],
        [ 1, 41, 53, ..., 46, 44, 59],
        [ 1, 40, 59, ...,  1, 46, 39]])>,
 <tf.Tensor: shape=(64, 100), dtype=int64, numpy=
 array([[ 1, 25, 53, ..., 56, 59, 58],
        [51, 43, 52, ...,  1, 21,  1],
        [52, 42,  1, ..., 52,  1, 47],
        ...,
        [ 1, 45, 53, ..., 53, 51, 43],
        [41, 53, 51, ..., 44, 59, 50],
        [40, 59, 58, ..., 46, 39, 57]])>)

In [26]:
model.layers[1](model.layers[0](a[0])).shape

TensorShape([64, 100, 1024])

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
target_example_batch[0,]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([33, 25, 21, 27, 10,  0, 23, 52, 53, 41, 49,  1, 39, 58,  1, 58, 46,
       43,  1, 45, 39, 58, 43,  2,  1, 27,  1, 46, 43, 39, 60, 43, 52, 57,
        2,  1, 31, 54, 39, 49, 43,  1, 63, 53, 59,  1, 52, 53, 58,  1, 58,
       46, 43, 57, 43,  0, 61, 53, 56, 42, 57,  1, 54, 50, 39, 47, 52,  6,
        1,  5, 31, 47, 56, 56, 39, 46,  6,  1, 49, 52, 53, 41, 49,  1, 51,
       43,  1, 46, 43, 56, 43,  6,  1, 56, 39, 54,  1, 51, 43,  1])>

In [31]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1736183


In [32]:
example_batch_loss

<tf.Tensor: shape=(64, 100), dtype=float32, numpy=
array([[4.1688004, 4.1738253, 4.189883 , ..., 4.1655064, 4.1577334,
        4.1774035],
       [4.1748977, 4.153286 , 4.152791 , ..., 4.170631 , 4.1789074,
        4.172943 ],
       [4.1779776, 4.172441 , 4.181379 , ..., 4.1718   , 4.1791797,
        4.167342 ],
       ...,
       [4.1651206, 4.188102 , 4.1744785, ..., 4.1627445, 4.1810064,
        4.16996  ],
       [4.1650343, 4.1759644, 4.1816955, ..., 4.196644 , 4.1764774,
        4.1847897],
       [4.1816435, 4.1823263, 4.187784 , ..., 4.183359 , 4.17641  ,
        4.1949773]], dtype=float32)>

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
EPOCHS=6

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], steps_per_epoch=50)

Epoch 1/6
50/50 [==============================] - 69s 1s/step - loss: 3.9013
Epoch 2/6
50/50 [==============================] - 67s 1s/step - loss: 2.5836
Epoch 3/6
50/50 [==============================] - 67s 1s/step - loss: 2.2994
Epoch 4/6
50/50 [==============================] - 28s 542ms/step - loss: 2.1748


In [39]:
tf.train.latest_checkpoint('./training_checkpoints/')

'./training_checkpoints/ckpt_2'

In [53]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [54]:
start_string=u"ROMEO: "

In [55]:
num_generate = 1000

In [56]:
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

In [58]:
text_generated = []
temperature = 1.0
model.reset_states()

In [60]:
model(input_eval).shape

TensorShape([1, 7, 65])

In [61]:
predictions = model(input_eval)
predictions = tf.squeeze(predictions, 0)
predictions = predictions / temperature
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

In [62]:
input_eval = tf.expand_dims([predicted_id], 0)

In [64]:
input_eval

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[20]], dtype=int32)>

In [48]:
predicted_id

20

In [173]:
input_eval = tf.expand_dims([predicted_id], 0)

In [174]:
input_eval

<tf.Tensor: id=87184, shape=(1, 1), dtype=int32, numpy=array([[39]], dtype=int32)>

In [65]:
text_generated.append(idx2char[predicted_id])

In [66]:
text_generated

['H']

In [69]:
model(input_eval).shape

TensorShape([1, 1, 65])

In [178]:
predictions = model(input_eval)

In [179]:
predictions = tf.squeeze(predictions, 0)

In [180]:
predictions = predictions / temperature

In [181]:
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

In [183]:
input_eval = tf.expand_dims([predicted_id], 0)

In [184]:
text_generated.append(idx2char[predicted_id])

In [185]:
text_generated

['a', 'd']